In [1]:
import pandas as pd
import numpy as np
import json
import datetime 

In [2]:
sourceCheckins = pd.read_csv('data/July/server/checkins.tsv', sep='\t')
checkins = pd.read_csv('data/July/server/extendedCheckinsHistory.tsv', sep='\t')
checkins1 = pd.read_csv('data/July/server/extendedCheckinsHistoryJune.tsv', sep='\t')
checkpoints = pd.read_csv('data/July/server/checkPoints.tsv', sep='\t')

In [13]:
len(np.unique(allCheckins.userId.tolist()))

647

In [14]:
len(allCheckins)

75690

In [11]:
allCheckins = allCheckins.drop_duplicates(subset = ['checkinId'])

In [10]:
allCheckins = pd.concat([checkins1, checkins])

In [3]:
allCheckins = pd.read_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t')

In [12]:
allCheckins.to_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t', encoding='utf-8')

In [11]:
sortedVenues = []
sortedVenuesIds = []
for venueId, group in allCheckins.groupby('venueId').agg('size').order(ascending = False).iteritems():
    sortedVenues.append({'venueId':venueId,
                        'group':group})
    sortedVenuesIds.append(venueId)

In [8]:
allCheckins[:2]

,Unnamed: 0,Unnamed: 0.1,categoryId,categoryName,checkinId,checkinsCount,createdAt,createdAtDate,lat,lng,name,tipCount,userId,usersCount,venueId
0,0,0,4bf58dd8d48988d1fd931735,Metro Station,558ac442498e91c19449d814,14585,1435157570,2015-06-24 17:52:50,55.782920,37.720699,Метро Семёновская (metro Semyonovskaya),26,74434491,3460,4bed6c2c9868a5933c555d46
1,1,1,4bf58dd8d48988d1d3941735,Vegetarian / Vegan Restaurant,558ababc498e7aa1a83dfb16,10096,1435155132,2015-06-24 17:12:12,55.758369,37.657153,Джаганнат,222,74434491,4972,4d8f05153bfef04d84a6a586


In [5]:
sortedVenues[:10]

[{'group': 874, 'venueId': '4bdd5f27f219c9b6bb751210'},
 {'group': 636, 'venueId': '4bfc9df283bbd13aa4d462c5'},
 {'group': 375, 'venueId': '4bc090ca920eb7132e14192c'},
 {'group': 344, 'venueId': '4e4aab0f62e13240b700e034'},
 {'group': 290, 'venueId': '4b62c18df964a52069512ae3'},
 {'group': 280, 'venueId': '4bc8760792b376b03767513a'},
 {'group': 254, 'venueId': '4bb3345942959c74d79d212c'},
 {'group': 233, 'venueId': '53110e9be4b03443dd848438'},
 {'group': 226, 'venueId': '4b63406af964a520a36d2ae3'},
 {'group': 220, 'venueId': '538c981b498e254777d000e8'}]

In [7]:
allCheckins[allCheckins.venueId == '4bdd5f27f219c9b6bb751210'][:1]['name']

3    ЦПКиО им. Горького / Gorky Park
Name: name, dtype: object

In [115]:
#'World Class Lite in top ten!, 520f117111d2a782f1211c3b'

In [15]:
groupedUsers = allCheckins.groupby('userId')
userVenues = []
for index, group in enumerate(groupedUsers):
        userVenues.append(group[1]['venueId'].tolist())

In [16]:
len(groupedUsers)

647

In [122]:
top50VenuesCheckins = allCheckins[allCheckins.venueId.isin(sortedVenuesIds[:50])]

In [127]:
for venueId in sortedVenuesIds[:50]:
    print top50VenuesCheckins[top50VenuesCheckins.venueId == venueId][:1]['name']

3    ЦПКиО им. Горького / Gorky Park
Name: name, dtype: object
485    Москва / Moscow
Name: name, dtype: object
348    Парк искусств «Музеон» / Muzeon Park
Name: name, dtype: object
639    ВДНХ (Выставка достижений народного хозяйства)
Name: name, dtype: object
187    ПКиО «Сокольники»
Name: name, dtype: object
1127    Международный аэропорт Домодедово / Domodedovo...
Name: name, dtype: object
317    ТРЦ «Европейский»
Name: name, dtype: object
807    Красная площадь / Red Square
Name: name, dtype: object
935    World Class Lite
Name: name, dtype: object
1094    Россия / Russia
Name: name, dtype: object
2    ТРК «Атриум» / Atrium Mall
Name: name, dtype: object
1335    Арбат / Arbat Street
Name: name, dtype: object
944    Музей-заповедник «Коломенское»
Name: name, dtype: object
630    Афимолл Сити / Afimall City
Name: name, dtype: object
901    Физика
Name: name, dtype: object
333    Нескучный сад
Name: name, dtype: object
687    Центральный детский магазин
Name: name, dtype: object
252 

In [12]:
from pymining import itemmining
relim_input = itemmining.get_relim_input(userVenues)
report = itemmining.relim(relim_input, min_support=2)
report

ImportError: No module named pymining

In [13]:
def load_dataset():
    "Load the sample dataset."
    return userVenues
 
 
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a ket of a dictionary.
    return map(frozenset, c1)
 
 
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1
 
    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data
 
 
def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList
 
 
def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = map(set, dataset)
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1
 
    return L, support_data

In [16]:
C1 = createC1(userVenues)

In [22]:
D = map(set, userVenues)

In [42]:
L1, support_data = scanD(D, C1, 0.01)

In [56]:
L, supportData = apriori(userVenues, minsupport=0.01)

In [27]:
frequentVenues = []
for itemset, support in zip(L, supportData):
    if len(itemset)>5:
        print support
        frequentVenueSet = []
        for item in itemset:
            frequentVenueSet.append(next(enumerate(item))[1])
        frequentVenues.append(frequentVenueSet)

frozenset(['526a9b4211d2be10cc62d215'])
frozenset(['4fca7a60e4b0d8a39128edcb'])


In [40]:
len(userVenues)*0.0015455950541

1.0000000000027

In [24]:
for venueId in frequentVenues[0]:
    print allCheckins[allCheckins.venueId == venueId][:1]['name']

630    Афимолл Сити / Afimall City
Name: name, dtype: object
3    ЦПКиО им. Горького / Gorky Park
Name: name, dtype: object
806    ТЦ «Охотный ряд»
Name: name, dtype: object
4735    Международный аэропорт Шереметьево / Sheremety...
Name: name, dtype: object
1335    Арбат / Arbat Street
Name: name, dtype: object
303    Патриаршие пруды / Patriarshiye Ponds
Name: name, dtype: object
20    Ленинградский вокзал / Leningradsky Railway Te...
Name: name, dtype: object
944    Музей-заповедник «Коломенское»
Name: name, dtype: object
317    ТРЦ «Европейский»
Name: name, dtype: object
1606    Парк Победы (Поклонная гора) / Victory Park
Name: name, dtype: object
1094    Россия / Russia
Name: name, dtype: object
2    ТРК «Атриум» / Atrium Mall
Name: name, dtype: object
1771    Александровский сад / Alexander Garden
Name: name, dtype: object
1127    Международный аэропорт Домодедово / Domodedovo...
Name: name, dtype: object
1820    Международный аэропорт Внуково / Vnukovo Inter...
Name: name, dtype:

In [94]:
frequentVenues

[['4d7b42a3f260a093113124ba',
  '4bdd5f27f219c9b6bb751210',
  '4be82d4488ed2d7fde85cb1d',
  '4b4d5b48f964a52055d126e3',
  '4bc09d91461576b055617a32',
  '4bb41c28f187a593f1e913f8',
  '4bc4f2f80191c9b6c7e0eab1',
  '4b827a36f964a52098d430e3',
  '4b63406af964a520a36d2ae3',
  '4bdc9716afe8c9b689235085',
  '53110e9be4b03443dd848438',
  '4b5086ccf964a520212627e3',
  '4bf7fa734a67c928e27624cf',
  '4b62c18df964a52069512ae3',
  '4b7563d3f964a520d8092ee3',
  '4b645287f964a52095aa2ae3',
  '4bfc9df283bbd13aa4d462c5',
  '4e4aab0f62e13240b700e034',
  '4bb3345942959c74d79d212c',
  '4b76c7bdf964a520a85e2ee3',
  '4bc8760792b376b03767513a',
  '4bc090ca920eb7132e14192c',
  '4e0776da1f6e2110338d2afc'],
 ['4bdd5f27f219c9b6bb751210',
  '4bdd5f27f219c9b6bb751210',
  '4bc090ca920eb7132e14192c',
  '4bdd5f27f219c9b6bb751210',
  '4bdd5f27f219c9b6bb751210',
  '4bdd5f27f219c9b6bb751210',
  '4bfc9df283bbd13aa4d462c5',
  '4bb3345942959c74d79d212c',
  '4bc090ca920eb7132e14192c',
  '4bdd5f27f219c9b6bb751210',
  '4bdd5f

In [45]:
supportData[frozenset(frequentVenues[1])]

KeyError: frozenset(['4bfc9df283bbd13aa4d462c5', '4bdd5f27f219c9b6bb751210', '4bb3345942959c74d79d212c', '4bc090ca920eb7132e14192c'])

In [57]:
def generateRules(L, support_data, min_confidence=0.7):
    """Create the association rules
    L: list of frequent item sets
    support_data: support data for those itemsets
    min_confidence: minimum confidence threshold
    """
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #print "freqSet", freqSet, 'H1', H1
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules
 
 
def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.7):
    "Evaluate the rule generated"
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            print freqSet - conseq, '--->', conseq, 'conf:', conf
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H
 
 
def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    "Generate a set of candidate rules"
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [ ]:
rules = generateRules(L, supportData, min_confidence=0.8)

In [98]:
venuesDict = {}
def venueNameFromId(venueId):
    if venueId not in venuesDict.keys():
        venuesDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['name']
    return venuesDict[venueId]

In [108]:
for rule in rules:
    simpleRule = 1
    if len(rule[0]) < 2:
        continue
        
    for venueId in enumerate(rule[0]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    simpleRule = 1
    
    for venueId in enumerate(rule[1]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    for venueId in enumerate(rule[0]):
        print venueNameFromId(venueId[1])
        
    print '--->'
    for venueId in enumerate(rule[1]):
        print venueNameFromId(venueId[1])
    print 'conf:', rule[2]
    print ''

ТРК «Атриум» / Atrium Mall
Дворцовая площадь / Palace Square
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Московский вокзал / Moskovsky Railway Station
conf: 0.888888888889

ТРЦ «Галерея» / Galeria Shopping Mall
ВДНХ (Выставка достижений народного хозяйства)
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Московский вокзал / Moskovsky Railway Station
conf: 0.846153846154

Парк искусств «Музеон» / Muzeon Park
ТРЦ «Галерея» / Galeria Shopping Mall
--->
ЦПКиО им. Горького / Gorky Park
Ленинградский вокзал / Leningradsky Railway Terminal
conf: 0.818181818182

Невский проспект / Nevsky Prospect
ВДНХ (Выставка достижений народного хозяйства)
--->
ЦПКиО им. Горького / Gorky Park
Дворцовая площадь / Palace Square
conf: 1.0

Ленинградский вокзал / Leningradsky Railway Terminal
Невский проспект / Nevsky Prospect
--->
Дворцовая площадь / Palace Square
Московский вокзал / Moskovsky Railway Station
conf: 1.0

Невский проспект / Nevsky Prospect
Московский вокзал / Moskovsky Ra

In [ ]:
#Using code for rules from here: http://aimotion.blogspot.ru/2013/01/machine-learning-and-data-mining.html

In [118]:
multiUserSortedVenues = filter(lambda x: x['group'] > 10, sortedVenues)

In [148]:
userVenueGroups = []
for venueId, group in allCheckins.groupby('venueId'):
    if len(group.groupby('userId'))>1:
        userVenueGroups.append(venueId)

In [207]:
#Make lists of users check-in venues
multiUserCheckins = allCheckins[allCheckins.venueId.isin(userVenueGroups)]
groupedUsers = multiUserCheckins.groupby('userId')
userVenuesList = []
userIds = []
for index, group in enumerate(groupedUsers):
        userVenuesList.append(group[1]['venueId'].tolist())
        userIds.append(group[0])

In [173]:
#Let's build some TF-ID vectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.cluster import MiniBatchKMeans

In [167]:
trainingList = []
for userList in userVenuesList:
    trainingList.append(' '.join(userList))

tfidf_vectorizer = TfidfVectorizer(vocabulary = userVenueGroups)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(trainingList)

In [176]:
from time import time
cocluster = SpectralCoclustering(n_clusters=20,
                                 svd_method='arpack', random_state=0)
kmeans = MiniBatchKMeans(n_clusters=20,
                         random_state=0)

print("Coclustering...")
start_time = time()
cocluster.fit(tfidf_matrix_train)
y_cocluster = cocluster.row_labels_
print("Done in {:.2f}s.".format(
    time() - start_time))

print("MiniBatchKMeans...")
start_time = time()
y_kmeans = kmeans.fit_predict(tfidf_matrix_train)
print("Done in {:.2f}s.".format(
    time() - start_time))

feature_names = tfidf_vectorizer.get_feature_names()

Coclustering...
Done in 0.20s.
MiniBatchKMeans...
Done in 0.11s.


In [189]:
sum(cocluster.columns_[0])

3912

In [210]:
clusters = x = [[] for i in range(20)]
for index, clustedId in enumerate(y_kmeans):
    clusters[clustedId].append(userIds[index])

In [228]:
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).intersection(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).union(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))

4
140
